In [91]:
import requests
import pandas as pd
import re
import folium
from geopy.geocoders import Nominatim

In [92]:
#I used Places API and could not get Cuisine type data in my search 
api_key = "USE YOUR OWN API KEY - I WILL NOT SHOW MINE "

indianapolis_latitude = 39.7684
indianapolis_longitude = -86.1581
keyword = "food truck"

base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

food_trucks = []

def fetch_data(page_token=None):
    params = {
        "query": keyword,
        "location": f'{indianapolis_latitude},{indianapolis_longitude}',
        "key": api_key,
        "pagetoken": page_token,
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    for result in data.get("results", []):
        name = result["name"]
        food_trucks.append({
            "Name": name
        })

    next_page_token = data.get("next_page_token")
    return next_page_token

next_page_token = fetch_data()
while next_page_token:
    next_page_token = fetch_data(next_page_token)

food_trucks_df = pd.DataFrame(food_trucks)

print(food_trucks_df)

                                    Name
0               The Night Owl Food Truck
1                           ClusterTruck
2               Tortas Guicho Food Truck
3               Pico de Gallo Food Truck
4           El Amigo Taqueria Food Truck
5                  Food truck Mil Amores
6                          T Street Eatz
7               El Buenavista Food Truck
8                      FELIX TACOS TRUCK
9                3 Marias Mex Food Truck
10                     Taqueria Mi Costa
11                  El Viejon Food Truck
12                          Tha Corn Man
13                 King gyros food truck
14             ROSE’S foodtruck Honduras
15                Super Tacos food truck
16                   Food Truck Don Luis
17                       Taco food truck
18                       Firehouse Dawgs
19        Antojitos Guzman food truck #1
20                     KG Slider Station
21         The Latin Flavor - Food Truck
22                           T'Days Jerk
23         Mi Li

In [93]:
pd.set_option('display.max_colwidth', None)
place_details = []

base_url_search = "https://maps.googleapis.com/maps/api/place/textsearch/json"
base_url_details = "https://maps.googleapis.com/maps/api/place/details/json"

for index, row in food_trucks_df.iterrows():
    place_name = row["Name"]

    params_search = {
        "query": place_name,
        "key": api_key,
    }
    response_search = requests.get(base_url_search, params=params_search)
    data_search = response_search.json()

    place_id = data_search["results"][0]["place_id"]
    params_details = {
        "place_id": place_id,
        "fields": "name,formatted_address,website,opening_hours",
        "key": api_key
    }
    response_details = requests.get(base_url_details, params=params_details)
    data_details = response_details.json()

    name = data_details["result"]["name"]
    address = data_details["result"]["formatted_address"]
    website = data_details["result"].get("website", "N/A")
    open_hours = data_details["result"].get("opening_hours", {}).get("weekday_text", "N/A")

    place_details.append({
        "Name": name,
        "Address": address,
        "Website": website,
        "Open Hours": open_hours
    })

place_details_df = pd.DataFrame(place_details)

In [94]:
place_details_df.head(20)

,Name,Address,Website,Open Hours
0,The Night Owl Food Truck,"South and, S Meridian St, Indianapolis, IN 46225, USA",http://mercersnightowl.com/,"[Monday: Closed, Tuesday: Closed, Wednesday: Closed, Thursday: 8:30 PM – 2:30 AM, Friday: 8:30 PM – 3:30 AM, Saturday: 8:30 PM – 3:30 AM, Sunday: 6:00 PM – 1:00 AM]"
1,ClusterTruck,"729 N Pennsylvania St, Indianapolis, IN 46204, USA",https://downtown-indy.clustertruck.com/menu?utm_source=google%20my%20business&utm_medium=search%20listing,"[Monday: 8:00 AM – 10:00 PM, Tuesday: 8:00 AM – 10:00 PM, Wednesday: 8:00 AM – 10:00 PM, Thursday: 8:00 AM – 10:00 PM, Friday: 8:00 AM – 11:00 PM, Saturday: 10:00 AM – 11:00 PM, Sunday: 10:00 AM – 9:00 PM]"
2,Tortas Guicho Food Truck,"S. East St and, Fletcher Ave, Indianapolis, IN 46204, USA",N/A,"[Monday: 8:00 AM – 2:00 PM, Tuesday: 8:00 AM – 2:00 PM, Wednesday: 8:00 AM – 2:00 PM, Thursday: 8:00 AM – 2:00 PM, Friday: 8:00 AM – 2:00 PM, Saturday: Closed, Sunday: Closed]"
3,Pico de Gallo Food Truck,"3014 Lafayette Rd #3000, Indianapolis, IN 46222, USA",https://www.facebook.com/peregrino1982/,"[Monday: Closed, Tuesday: Closed, Wednesday: 5:00 PM – 1:00 AM, Thursday: 5:00 PM – 1:00 AM, Friday: 5:00 PM – 4:00 AM, Saturday: 5:00 PM – 4:00 AM, Sunday: 5:00 PM – 1:00 AM]"
4,El Amigo Taqueria Food Truck,"3356 W 16th St, Indianapolis, IN 46222, USA",N/A,"[Monday: 12:00 PM – 12:00 AM, Tuesday: 12:00 PM – 12:00 AM, Wednesday: 12:00 PM – 12:00 AM, Thursday: 12:00 PM – 12:00 AM, Friday: 12:00 PM – 12:00 AM, Saturday: 12:00 PM – 12:00 AM, Sunday: 12:00 PM – 12:00 AM]"
5,Food truck Mil Amores,"5034 E Raymond St, Indianapolis, IN 46203, USA",N/A,N/A
6,T Street Eatz,"City Market, 222 E Market St, Indianapolis, IN 46204, USA",http://www.tstreeteatzindy.com/,"[Monday: 11:30 AM – 4:00 PM, Tuesday: 11:30 AM – 4:00 PM, Wednesday: 11:30 AM – 8:00 PM, Thursday: 11:30 AM – 4:00 PM, Friday: 11:30 AM – 4:00 PM, Saturday: 11:30 AM – 4:00 PM, Sunday: Closed]"
7,El Buenavista Food Truck,"5210 English Ave, Indianapolis, IN 46219, USA",N/A,"[Monday: 1:59 PM – 4:59 AM, Tuesday: 1:59 PM – 4:59 AM, Wednesday: 1:59 PM – 4:59 AM, Thursday: 1:59 PM – 4:59 AM, Friday: 1:59 PM – 4:59 AM, Saturday: 1:59 PM – 4:59 AM, Sunday: 1:59 PM – 4:59 AM]"
8,FELIX TACOS TRUCK,"3145 W Washington St, Indianapolis, IN 46222, USA",N/A,"[Monday: 5:00 PM – 12:00 AM, Tuesday: 5:00 PM – 12:00 AM, Wednesday: 5:00 PM – 12:00 AM, Thursday: 5:00 PM – 12:00 AM, Friday: 5:00 PM – 2:00 AM, Saturday: 5:00 PM – 2:00 AM, Sunday: 5:00 PM – 12:00 AM]"
9,3 Marias Mex Food Truck,"3205 E Washington St, Indianapolis, IN 46201, USA",N/A,"[Monday: 6:00 – 10:00 PM, Tuesday: 6:00 – 10:00 PM, Wednesday: 6:00 – 10:00 PM, Thursday: 6:00 – 10:00 PM, Friday: Closed, Saturday: 6:00 – 10:00 PM, Sunday: Closed]"


In [95]:
def format_open_hours(hours_list):
    if hours_list == "N/A":
        return "N/A"
    
    formatted_hours = ', '.join(hours_list)
    formatted_hours = re.sub(r'\[|\]', '', formatted_hours)
    return formatted_hours

In [96]:
df = place_details_df.copy()

df['Open Hours'] = df['Open Hours'].apply(format_open_hours)

print(df)

                                    Name  \
0               The Night Owl Food Truck   
1                           ClusterTruck   
2               Tortas Guicho Food Truck   
3               Pico de Gallo Food Truck   
4           El Amigo Taqueria Food Truck   
5                  Food truck Mil Amores   
6                          T Street Eatz   
7               El Buenavista Food Truck   
8                      FELIX TACOS TRUCK   
9                3 Marias Mex Food Truck   
10                     Taqueria Mi Costa   
11                  El Viejon Food Truck   
12                          Tha Corn Man   
13                 King gyros food truck   
14                    Rose Bistro & Cafe   
15                 Super Taco Express #1   
16                   Food Truck Don Luis   
17         Caseras sabor real food truck   
18                       Firehouse Dawgs   
19        Antojitos Guzman food truck #1   
20                     KG Slider Station   
21         The Latin Flavor - Fo

In [97]:
#parsing the open hours to be better
def process_open_hours(hours, day):
    for part in hours.split(', '):
        if part.startswith(day):
            hours_str = part.split(": ")[1].replace("Closed, ", "").replace("Closed", "N/A")
            return hours_str
    return 'N/A'  

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for day in days_of_week:
    df[f'{day} Open Hours'] = df['Open Hours'].apply(lambda x: process_open_hours(x, day))

df = df.drop(columns=['Open Hours'])

df.to_excel('XTernDataScienceFoodTruck.xlsx', index=False)

print(df)

                                    Name  \
0               The Night Owl Food Truck   
1                           ClusterTruck   
2               Tortas Guicho Food Truck   
3               Pico de Gallo Food Truck   
4           El Amigo Taqueria Food Truck   
5                  Food truck Mil Amores   
6                          T Street Eatz   
7               El Buenavista Food Truck   
8                      FELIX TACOS TRUCK   
9                3 Marias Mex Food Truck   
10                     Taqueria Mi Costa   
11                  El Viejon Food Truck   
12                          Tha Corn Man   
13                 King gyros food truck   
14                    Rose Bistro & Cafe   
15                 Super Taco Express #1   
16                   Food Truck Don Luis   
17         Caseras sabor real food truck   
18                       Firehouse Dawgs   
19        Antojitos Guzman food truck #1   
20                     KG Slider Station   
21         The Latin Flavor - Fo

In [98]:
#Picking and mapping out my favorite routes - I did this on my own for what I like and used Google Maps
#to route the plan
foodie_plan = pd.DataFrame(columns=["Time", "Location Name", "Address", "Cuisine", "Travel Time", "Travel Distance", "Transportation Type"])

day_1 = pd.DataFrame({
    "Time": ["10:00 AM", "3:00 PM", "8:30 PM"],
    "Location Name": ["ClusterTruck", "T Street Eatz", "Tha Corn Man"],
    "Address": ["729 N Pennsylvania St, Indianapolis, IN 46204, USA", "City Market, 222 E Market St, Indianapolis, IN 46204, USA", "2442 Central Ave, Indianapolis, IN 46205, USA"],
    "Cuisine": ["Cuisine 1", "Cuisine 2", "Corn"],
    "Travel Time": ["20 min", "18 min", "8 min"],
    "Travel Distance": ["0.7 miles", "0.8 miles", "2.5 miles"],
    "Transportation Type": ["Walk", "Walk", "Car"]
})

day_2 = pd.DataFrame({
    "Time": ["12:00 PM", "5:00 PM", "10:00 PM"],
    "Location Name": ["El Amigo Taqueria Food Truck", "Taqueria Mi Costa", "FELIX TACOS TRUCK"],
    "Address": ["3356 W 16th St, Indianapolis, IN 46222, USA", "1430 W Washington St, Indianapolis, IN 46222, USA", "3145 W Washington St, Indianapolis, IN 46222, USA"],
    "Cuisine": ["Taqueria", "Taqueria", "Tacos"],
    "Travel Time": ["13 min", "7 min", "4 min"],
    "Travel Distance": ["4.1 miles", "3 miles", "1.4 miles"],
    "Transportation Type": ["Car", "Car", "Car"]
})

foodie_plan = pd.concat([foodie_plan, day_1], ignore_index=True)
foodie_plan = pd.concat([foodie_plan, day_2], ignore_index=True)

print(foodie_plan)

       Time                 Location Name  \
0  10:00 AM                  ClusterTruck   
1   3:00 PM                 T Street Eatz   
2   8:30 PM                  Tha Corn Man   
3  12:00 PM  El Amigo Taqueria Food Truck   
4   5:00 PM             Taqueria Mi Costa   
5  10:00 PM             FELIX TACOS TRUCK   

                                                     Address    Cuisine  \
0         729 N Pennsylvania St, Indianapolis, IN 46204, USA  Cuisine 1   
1  City Market, 222 E Market St, Indianapolis, IN 46204, USA  Cuisine 2   
2              2442 Central Ave, Indianapolis, IN 46205, USA       Corn   
3                3356 W 16th St, Indianapolis, IN 46222, USA   Taqueria   
4          1430 W Washington St, Indianapolis, IN 46222, USA   Taqueria   
5          3145 W Washington St, Indianapolis, IN 46222, USA      Tacos   

  Travel Time Travel Distance Transportation Type  
0      20 min       0.7 miles                Walk  
1      18 min       0.8 miles                Walk  
2 

In [99]:
# I could not get the geolocator to extract lat and long values from the addresses so I had to hard code the values in for my
#planned trip to display the info using Python tools
location_df = foodie_plan.copy()
data = [
    {"Name": "ClusterTruck", "Longitude": -86.1583, "Latitude": 39.7726, "Day": 1},
    {"Name": "T Street Eatz", "Longitude": -86.1524, "Latitude": 39.7752, "Day": 1},
    {"Name": "Tha Corn Man", "Longitude": -86.1496, "Latitude": 39.8017, "Day": 1},
    {"Name": "El Amigo Taqueria Food Truck", "Longitude": -86.2162, "Latitude": 39.7885, "Day": 2},
    {"Name": "Taqueria Mi Costa", "Longitude": -86.1857, "Latitude": 39.7726, "Day": 2},
    {"Name": "FELIX TACOS TRUCK", "Longitude": -86.2123, "Latitude": 39.7633, "Day": 2},
]
df = pd.DataFrame(data)

location_df["Longitude"] = df["Longitude"]
location_df["Latitude"] = df["Latitude"]
location_df["Day"] = df["Day"]

location_df.to_csv("foodie_plan.csv", index=False)

print(location_df)

       Time                 Location Name  \
0  10:00 AM                  ClusterTruck   
1   3:00 PM                 T Street Eatz   
2   8:30 PM                  Tha Corn Man   
3  12:00 PM  El Amigo Taqueria Food Truck   
4   5:00 PM             Taqueria Mi Costa   
5  10:00 PM             FELIX TACOS TRUCK   

                                                     Address    Cuisine  \
0         729 N Pennsylvania St, Indianapolis, IN 46204, USA  Cuisine 1   
1  City Market, 222 E Market St, Indianapolis, IN 46204, USA  Cuisine 2   
2              2442 Central Ave, Indianapolis, IN 46205, USA       Corn   
3                3356 W 16th St, Indianapolis, IN 46222, USA   Taqueria   
4          1430 W Washington St, Indianapolis, IN 46222, USA   Taqueria   
5          3145 W Washington St, Indianapolis, IN 46222, USA      Tacos   

  Travel Time Travel Distance Transportation Type  Longitude  Latitude  Day  
0      20 min       0.7 miles                Walk   -86.1583   39.7726    1  
1 

In [100]:
indianapolis_latitude = 39.7684
indianapolis_longitude = -86.1581

colors = ["blue", "red"]

combined_map = folium.Map(location=[indianapolis_latitude, indianapolis_longitude], zoom_start=12)

starting_location = (indianapolis_latitude, indianapolis_longitude)
folium.Marker(location=starting_location, icon=folium.Icon(color="green"), popup="Starting Location").add_to(combined_map)

previous_location = starting_location
current_day = None

for index, row in location_df.iterrows():
    location_coordinates = (row["Latitude"], row["Longitude"])
    location_name = row["Location Name"]
    travel_time = row["Travel Time"]
    travel_distance = row["Travel Distance"]
    location_type = row["Transportation Type"]  
    day = row["Day"] 

    if day != current_day:
        previous_location = starting_location
        current_day = day

    marker = folium.Marker(
        location=location_coordinates,
        popup=f"Day {day}<br>{location_name}<br>Type: {location_type}<br>Travel Time: {travel_time}<br>Travel Distance: {travel_distance}",
    )

    marker.add_to(combined_map)
    folium.PolyLine([previous_location, location_coordinates], color=colors[day - 1]).add_to(combined_map)
    previous_location = location_coordinates

combined_map.save("twoDayFoodiePlan.html")


In [90]:
#I could not get the geolocator to work... I got accessed denied and am skeptical using API keys that require more pay after the 
# places... but this was the code I was trying to model

foodie_plan = pd.read_csv("foodie_plan.csv")

geolocator = Nominatim(user_agent="foodie_route")
indianapolis_latitude = 39.7684
indianapolis_longitude = -86.1581
map = folium.Map(location=[indianapolis_latitude, indianapolis_longitude], zoom_start=12)

previous_location = None

for index, row in foodie_plan.iterrows():
    address = row["Address"]

    location = geolocator.geocode(address)

    if location is not None:
        location_coordinates = [location.latitude, location.longitude]
    

        marker = folium.Marker(
            location=location_coordinates,
            popup=row["Location Name"],
        )
        marker.add_to(map)

        if previous_location:
            folium.PolyLine([previous_location, location_coordinates], color="blue").add_to(map)

        previous_location = location_coordinates

map.save("foodie_route.html")

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=City+Market%2C+222+E+Market+St%2C+Indianapolis%2C+IN+46204%2C+USA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))